In [1]:
import numpy as np
import scipy as sp

import ast
import os
import re

from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian
import quchem.Misc_functions.conversion_scripts as conv_scr 
from copy import deepcopy
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Get_reduced_H_matrix_SeqRot, SeqRot_linalg_Energy
from openfermion import qubit_operator_sparse

import pickle
import datetime

#######
import sys




In [2]:
working_dir = os.path.dirname(os.getcwd())
Analysis_dir = os.path.join(working_dir, 'Analysis')
full_H_results_dir = os.path.join(Analysis_dir, 'SeqRot_LCU_script_A_results')

In [3]:
###### IMPORT INITIAL RESULTS

## import SeqRot results
myriad_SeqRot_results = {}
for filename in os.listdir(full_H_results_dir):
    if (filename.endswith('.pickle') and filename.startswith('SeqRot_CS_VQE_exp')):
        file_path = os.path.join(full_H_results_dir, filename) 
        mol_name = filename[43:-8]
        with open(file_path,'rb') as infile:
            data = pickle.load(infile)
        myriad_SeqRot_results[mol_name] = data

In [44]:

######## take commandline arguement to run in parallel
AC_set_index  = 10 # minus one as array script idexes from 1
mol_key = 'C1-O1_STO-3G_singlet'

check_reduction_SeqRot = False

if mol_key not in myriad_SeqRot_results.keys():
    raise ValueError('molecule key not correct')

In [45]:
########
## import AC_sets results

AC_sets_dir_name = 'AC_sets_SeqRot'
AC_dir = os.path.join(working_dir, AC_sets_dir_name)
input_AC_file_path = os.path.join(AC_dir, mol_key + '.pickle') # AC of given molecule


with open(input_AC_file_path,'rb') as infile:
    all_anti_commuting_sets_SeqRot = pickle.load(infile)

anti_commuting_sets_SeqRot = all_anti_commuting_sets_SeqRot[AC_set_index]['AC_sets']
ground_state_ket = all_anti_commuting_sets_SeqRot[AC_set_index]['ground_state']

In [46]:
## Get Energy

if anti_commuting_sets_SeqRot:
    ### SeqRot
    all_zero_Ps_index_dict = {set_key: 0 for set_key in anti_commuting_sets_SeqRot}

    H_SeqRot_dict = myriad_SeqRot_results[mol_key][AC_set_index]['H']
    n_qubits = len(list(H_SeqRot_dict.keys())[0])

#     H_sparse = Get_reduced_H_matrix_SeqRot(anti_commuting_sets_SeqRot,
#                                      all_zero_Ps_index_dict,
#                                      n_qubits,
#                                      atol=1e-8,
#                                      rtol=1e-05,
#                                      check_reduction=check_reduction_SeqRot)
    
    
#     E_SeqRot = SeqRot_linalg_Energy(anti_commuting_sets_SeqRot,
#                                  all_zero_Ps_index_dict,
#                                  n_qubits,
#                                  atol=1e-8,
#                                  rtol=1e-05,
#                                  check_reduction=check_reduction_SeqRot)

#     AC_set_and_Energy_output = {'AC_sets': anti_commuting_sets_SeqRot,
#                                                            'E':E_SeqRot}
    
    
    ########
    
    
#     denisty_mat = np.outer(ground_state_ket, ground_state_ket)
#     E_SeqRot = np.trace(denisty_mat@H_sparse)

#     AC_set_and_Energy_output = {'AC_sets': anti_commuting_sets_SeqRot,
#                                                            'E':E_SeqRot}
# else:
#     # only non-contextual problem
#     AC_set_and_Energy_output = {'AC_sets': anti_commuting_sets_SeqRot,
#                                                            'E':myriad_SeqRot_results[mol_key][AC_set_index]['E']}    




In [ ]:
%timeit SeqRot_linalg_Energy(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)

In [ ]:
%timeit SeqRot_linalg_Energy_FAST(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)

In [ ]:
E1= SeqRot_linalg_Energy(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)
E2= SeqRot_linalg_Energy_FAST(anti_commuting_sets_SeqRot,all_zero_Ps_index_dict,n_qubits,atol=1e-8,rtol=1e-05,check_reduction=check_reduction_SeqRot)

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def test1():
    anti_commuting_sets = anti_commuting_sets_SeqRot
    S_key_dict = all_zero_Ps_index_dict
    N_Qubits = n_qubits
    atol=1e-8
    rtol=1e-05
    check_reduction=False


    H_single_terms = QubitOperator()
    gammal_Rdag_P_R_terms = QubitOperator()
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            H_single_terms += AC_set[0]
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)

            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)

            R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
            R_dag_P_R = Ps * R_S
            gammal_Rdag_P_R_terms += gamma_l*R_dag_P_R

    all_symbolic_ops = H_single_terms + gammal_Rdag_P_R_terms
    reduced_H_matrix = fast_qubit_operator_sparse(all_symbolic_ops, N_Qubits)

In [ ]:
def test2():
    anti_commuting_sets = anti_commuting_sets_SeqRot
    S_key_dict = all_zero_Ps_index_dict
    N_Qubits = n_qubits
    atol=1e-8
    rtol=1e-05
    check_reduction=False


    reduced_H_matrix = csr_matrix((2 ** N_Qubits, 2 ** N_Qubits), dtype=complex)
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            CiPi = AC_set[0]
            CiPi_matrix = fast_qubit_operator_sparse(CiPi, N_Qubits)
            reduced_H_matrix+=CiPi_matrix
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)

            R_S_matrix = fast_qubit_operator_sparse(R_S, N_Qubits)
            P_S_matrix = fast_qubit_operator_sparse(Ps, N_Qubits)
            Rdag_P_R= R_S_matrix.conj().T @ P_S_matrix @ R_S_matrix
            reduced_H_matrix+=gamma_l*Rdag_P_R


In [ ]:
def test3():
    anti_commuting_sets = anti_commuting_sets_SeqRot
    S_key_dict = all_zero_Ps_index_dict
    N_Qubits = n_qubits
    atol=1e-8
    rtol=1e-05
    check_reduction=False


    reduced_H_matrix = csr_matrix((2 ** N_Qubits, 2 ** N_Qubits), dtype=complex)
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            CiPi = AC_set[0]
            CiPi_matrix = fast_qubit_operator_sparse(CiPi, N_Qubits)
            reduced_H_matrix+=CiPi_matrix
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)
            
            R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
            R_dag_P_R_matrix = fast_qubit_operator_sparse(R_dag_P_R, N_Qubits)
            reduced_H_matrix+=gamma_l*R_dag_P_R_matrix


In [ ]:
%timeit test1()

In [ ]:
%timeit test2()

In [ ]:
%timeit test3()

In [ ]:
%timeit R_S_matrix.conj().T

In [ ]:
%timeit fast_qubit_operator_sparse(hermitian_conjugated(R_S), N_Qubits)

In [ ]:
len(list(all_symbolic_ops)) * 0.5

In [ ]:
# ####### SAVE OUTPUT details
# unique_file_time = datetime.datetime.now().strftime('%Y%b%d-%H%M%S%f')
# working_directory = os.getcwd()
# output_dir =os.path.join(working_directory, mol_key)

# # Create target Directory if it doesn't exist
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)


# # save file
# file_name1 = 'AC_set_and_Energy_output_set_key_{}.pickle'.format(AC_set_index)
# file_out1=os.path.join(output_dir, file_name1)

# ####### SAVE OUTPUT
# with open(file_out1, 'wb') as outfile:
#     pickle.dump(AC_set_and_Energy_output, outfile)


# print('pickle files dumped at: {}'.format(file_out1))

# print('end time: {}'.format(datetime.datetime.now().strftime('%Y%b%d-%H%M%S%f')))

In [ ]:
from quchem.Misc_functions.Misc_functions import fast_qubit_operator_sparse
from openfermion import QubitOperator, hermitian_conjugated
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import  Get_Xsk_op_list, Get_Rsl_matrix_as_qubitops
from scipy.linalg import eigh
from scipy.sparse.linalg import eigsh

def SeqRot_linalg_Energy_FAST(anti_commuting_sets, S_key_dict, N_Qubits, atol=1e-8, rtol=1e-05, check_reduction=False):
    """
    Function giving ground state energy of Hamiltonian given as a dictionary of anti-commuting sets. Note this uses symbolic operators and only builds sparse matrix once.


    Args:
        anti_commuting_sets (dict): dictionary of int keys with list of anti commuting QubitOperators sets
        S_key_dict(dict): dictionary keys match that of anti_commuting_sets. Value gives index of P_s operator
        N_Qubits(int): number of qubits

    returns:
        FCI_Energy(float): Ground state energy

    """
    # TODO: could return reduced_H_matrix sparse matrix!


    H_single_terms = QubitOperator()
    gammal_Rdag_P_R_terms = QubitOperator()
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            H_single_terms += AC_set[0]
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)
            R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
            gammal_Rdag_P_R_terms += gamma_l*R_dag_P_R

    all_symbolic_ops = H_single_terms + gammal_Rdag_P_R_terms
    reduced_H_matrix = fast_qubit_operator_sparse(all_symbolic_ops, N_Qubits)
#     reduced_H_matrix = qubit_operator_sparse(all_symbolic_ops, n_qubits=N_Qubits)
    # eig_values, eig_vectors = sparse_eigs(reduced_H_matrix)
    if reduced_H_matrix.shape[0]<=64:
        eig_values, eig_vectors = eigh(reduced_H_matrix.todense()) # NOT sparse!
    else:
        eig_values, eig_vectors = eigsh(reduced_H_matrix, k=1, which='SA') # < solves eigenvalue problem for a complex Hermitian matrix.
    FCI_Energy = min(eig_values)
    return FCI_Energy

In [ ]:
H_sparse = Get_reduced_H_matrix_SeqRot_matrix_FAST(anti_commuting_sets_SeqRot,
                                 all_zero_Ps_index_dict,
                                 n_qubits,
                                 atol=1e-8,
                                 rtol=1e-05,
                                 check_reduction=check_reduction_SeqRot)

In [ ]:
ground_state_ket.shape

In [ ]:
denisty_mat = np.outer(ground_state_ket, ground_state_ket)
np.trace(H_sparse @ denisty_mat)

In [ ]:
ground_state_ket.conj().T @ H_sparse @ ground_state_ket

In [ ]:
np.around(np.array([0.4,0.00005]),3)

In [ ]:
from scipy.sparse import csc_matrix

In [ ]:
thresh = 10
sparse_row = csr_matrix(np.around(ground_state_ket,thresh).reshape([ground_state_ket.shape[0],1]), dtype=complex)
sparse_col = csc_matrix(np.around(ground_state_ket,thresh).reshape([ground_state_ket.shape[0],1]), dtype=complex)


In [ ]:
# %timeit sparse_row.conj().T @ H_sparse @ sparse_row

In [ ]:
%timeit sparse_row.conj().T @ H_sparse @ sparse_row

In [ ]:
%timeit sparse_col.conj().T @ H_sparse @ sparse_col

In [ ]:
%timeit ground_state_ket.conj().T @ H_sparse @ ground_state_ket

In [ ]:
eig_values, eig_vectors = eigh(H_sparse.todense()) # NOT sparse!#
min(eig_values)

In [ ]:
E = sparse_col.conj().T @ H_sparse @ sparse_col
E.todense().item(0)

In [ ]:
min(eig_values) - E.todense().item(0)

In [ ]:
H_sparse = Get_reduced_H_matrix_SeqRot(anti_commuting_sets_SeqRot,
                                 all_zero_Ps_index_dict,
                                 n_qubits,
                                 atol=1e-8,
                                 rtol=1e-05,
                                 check_reduction=check_reduction_SeqRot)

In [ ]:
from scipy.sparse import csc_matrix

In [ ]:
decimal_place_threshold=10
sparse_ket = csc_matrix(np.around(ground_state_ket,decimal_place_threshold).reshape([ground_state_ket.shape[0],1]), dtype=complex)

In [ ]:
E_SeqRot = sparse_ket.conj().T @ H_sparse @ sparse_ket
E_SeqRot.todense().item(0)

In [7]:
from functools import reduce
from scipy.sparse import csr_matrix
from scipy.sparse import kron
import numpy as np
from scipy.sparse.linalg import expm, eigsh

from quchem.Misc_functions.Misc_functions import sparse_allclose

from openfermion.ops import QubitOperator
from openfermion.linalg import qubit_operator_sparse
from openfermion import hermitian_conjugated
from scipy.sparse import csc_matrix
from scipy.linalg import eigh
from openfermion.utils import hermitian_conjugated
from quchem.Misc_functions.Misc_functions import fast_qubit_operator_sparse

from tqdm.notebook import tqdm
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import  Get_Xsk_op_list, Get_Rsl_matrix_as_qubitops

def Get_reduced_H_matrix_SeqRot(anti_commuting_sets, S_key_dict, N_Qubits, atol=1e-8, rtol=1e-05, check_reduction=False):
    """
    Function giving ground state energy of Hamiltonian given as a dictionary of anti-commuting sets. Note this uses symbolic operators and only builds sparse matrix once.


    Args:
        anti_commuting_sets (dict): dictionary of int keys with list of anti commuting QubitOperators sets
        S_key_dict(dict): dictionary keys match that of anti_commuting_sets. Value gives index of P_s operator
        N_Qubits(int): number of qubits

    returns:
        reduced_H_matrix(scipy.sparse_matrix): sparse Hamiltonian matrix after unitary partitioning via SeqRot

    """
    H_single_terms = QubitOperator()
    gammal_Rdag_P_R_terms = QubitOperator()
    for key in tqdm(anti_commuting_sets):
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            H_single_terms += AC_set[0]
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)


            R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)
#             R_dag_P_R = hermitian_conjugated(R_S) * Ps * R_S
#             gammal_Rdag_P_R_terms += gamma_l*R_dag_P_R

    all_symbolic_ops = H_single_terms + gammal_Rdag_P_R_terms
    # reduced_H_matrix = qubit_operator_sparse(all_symbolic_ops, n_qubits=N_Qubits)
#     reduced_H_matrix = fast_qubit_operator_sparse(all_symbolic_ops, N_Qubits)
    return all_symbolic_ops

In [ ]:
R_S = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)

In [ ]:
R_sk_list = []
for X_sk_Op, theta_sk in X_sk_theta_sk_list:
    op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
    R_sk_list.append(op)
# R_sk_list

In [ ]:
R_sk_list

In [ ]:
reversed_list = R_sk_list[::-1]
running_ops = reversed_list[0]
for P_op in reversed_list[1:]:
    running_ops = running_ops*P_op
running_ops  

# R_S_q_ops = reduce(lambda x,y: x*y, R_sk_list[::-1])

In [ ]:
len(list(R_S_q_ops))

In [ ]:
len(X_sk_theta_sk_list)

In [ ]:
R_sk_MATRIX_list = []
for X_sk_Op, theta_sk in X_sk_theta_sk_list:
    op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
    mat_op = qubit_operator_sparse(op,n_qubits=n_qubits)
    R_sk_MATRIX_list.append(mat_op)
# R_sk_list

In [ ]:
selected_matrix = R_sk_MATRIX_list[0]
from scipy.sparse import find

r1, c1, v1 = find(selected_matrix)  # row indices, column indices, and values of the nonzero matrix entries
r1.shape

In [ ]:
# Rs_l_matrix = reduce(np.dot, R_sk_MATRIX_list[::-1])

In [ ]:
R_sk_MATRIX_list = []
for X_sk_Op, theta_sk in tqdm(X_sk_theta_sk_list):
    pauliword_X_sk_MATRIX = qubit_operator_sparse(QubitOperator(list(X_sk_Op.terms.keys())[0], -1j),
                                                  n_qubits=n_qubits)
    const_X_sk = list(X_sk_Op.terms.values())[0]

    R_sk_MATRIX_list.append(expm(pauliword_X_sk_MATRIX * theta_sk / 2 * const_X_sk))
# Rs_l_matrix = reduce(np.dot, R_sk_MATRIX_list[::-1])  # <- note reverse order!

In [ ]:
# R_S_Dag = hermitian_conjugated(R_S_q_ops)
# R_dag_P_R = R_S_Dag * Ps * R_S_q_ops

# R = fast_qubit_operator_sparse(R_S_q_ops, n_qubits)
R = qubit_operator_sparse(R_S_q_ops, n_qubits=n_qubits)
R_dag =  R.conj().T

In [ ]:
reversed_list = R_sk_list[::-1]
new_term=QubitOperator()
for ind, P_op in enumerate(reversed_list[:-1]):
    I_term_A, P_term_A = R_sk_list[ind]
    I_term_B, P_term_B = R_sk_list[ind+1]
    
    new_term += (I_term_A*I_term_B) + (I_term_A*P_term_B) + (P_term_A*I_term_B) + (P_term_A*I_term_B) 

new_term

In [ ]:
R_S_q_ops = reduce(lambda x,y: x*y, R_sk_list[::-1])
R_S_q_ops

In [ ]:
I_term, P_term = R_sk_list[0]
P_term

In [ ]:
X_sk_theta_sk_list

In [ ]:
def apply_Udag_P_U(X_sk_theta_sk_list, Pauli_S):
    
    running_term = QubitOperator()
    for P_word, theta_sk in X_sk_theta_sk_list:
        if P_word*Pauli_S==Pauli_S*P_word:
            # R_dag P R = I P
            running_term+= Pauli_S
        else:
            R = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
            running_term+= np.cos(theta_sk)*Pauli_S + 1j*np.sin(theta_sk)**
    
    return running_term
    

In [ ]:
def apply_rotation(rotation,p):
    
    out = {}
    
    if not commute(rotation[1],p):
        if rotation[0] == 'pi/2':
            q = pauli_mult(rotation[1],p)
            out[q[0]] = (1j*q[1]).real
    
        else:
            out[p] = np.cos(rotation[0])
            q = pauli_mult(rotation[1],p)
            out[q[0]] = (1j*q[1]*np.sin(rotation[0])).real
            
    else:
            out[p] = 1.
    
    return out

In [ ]:
H_sparse = Get_reduced_H_matrix_SeqRot(anti_commuting_sets_SeqRot,
                                 all_zero_Ps_index_dict,
                                 n_qubits,
                                 atol=1e-8,
                                 rtol=1e-05,
                                 check_reduction=check_reduction_SeqRot)

In [ ]:
X_sk_theta_sk_list


In [ ]:
    rotations = []
    
    # if there are cliques...
    if fn_form[1] > 0:
        # rotations to map A to a single Pauli (to be applied on left)
        for i in range(1,fn_form[1]):
            theta = np.arctan2(ep_state[1][i],np.sqrt(sum([ep_state[1][j]**2 for j in range(i)])))
            if i == 1 and ep_state[1][0] < 0:
                theta = np.pi - theta
            generator = pauli_mult(model[1][0],model[1][i])
            sgn = generator[1].imag
            rotations.append( [sgn*theta, generator[0]] )
    
        # rotations to diagonalize G union with the new A
        GuA = deepcopy(model[0] + [model[1][0]])
        ep_state_trans = deepcopy(ep_state[0] + [1])

In [ ]:
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Normalise_Clique

out = Normalise_Clique(anti_commuting_sets_SeqRot[4])
PWords = out['PauliWords']
PWords

In [ ]:
def Get_Xsk_op_list_NEW(anti_commuting_set, S_index, N_Qubits, check_reduction=False, atol=1e-8, rtol=1e-05):
    """
    Function to give all X_sk operators from a given anti_commuting set and S_index

    Args:
        anti_commuting_set(list): list of anti commuting QubitOperators
        S_index(int): index for Ps in anti_commuting_set list

    returns:
        X_sk_theta_sk_list(list): list of tuples containing X_sk QubitOperator and Theta_sk value
        normalised_FULL_set(dict): 'PauliWords' key gives NORMALISED terms that make up anti_commuting set
                                    'gamma_l' key gives normalization term
        Ps (QubitOperator): Pauli_S operator with cofactor of 1!
        gamma_l (float): normalization term

    """
    # 𝛾_𝑙 ∑ 𝛽_𝑗 𝑃_𝑗
    normalised_FULL_set = Normalise_Clique(anti_commuting_set)
    gamma_l = normalised_FULL_set['gamma_l']

    # ∑ 𝛽_𝑗 𝑃_𝑗
    norm_FULL_set = normalised_FULL_set['PauliWords'].copy()
    Pauli_S = norm_FULL_set.pop(S_index)  # removed from list!

    PauliStr_Ps, beta_S = tuple(*Pauli_S.terms.items())
    Ps = QubitOperator(PauliStr_Ps, 1) # new constant of 1

    X_sk_theta_sk_list = []
    for i, BetaK_Pk in enumerate(norm_FULL_set):
        
        Pk, BetaK = zip(*list(BetaK_Pk.terms.items()))
        denominator = sum(list(Pword.terms.values())[0]**2 for Pword in norm_FULL_set[:(i+1)])
        theta_sk = np.arctan2(BetaK, np.sqrt(denominator))
        
        Pk, BetaK = zip(*list(BetaK_Pk.terms.items()))
        X_sk = 1j * Ps * QubitOperator(Pk[0], 1) # new constant of 1

        if i == 0 and beta_S < 0:
            theta_sk = np.pi - theta_sk
        
        X_sk_theta_sk_list.append((X_sk, theta_sk))
        

    ### check transformation - SYMBOLIC (cheaper than above)!
    if check_reduction:
        R_sk_OP_list = []
        for X_sk_Op, theta_sk in X_sk_theta_sk_list:
            op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
            R_sk_OP_list.append(op)

        R_S_op = reduce(lambda x,y: x*y, R_sk_OP_list[::-1])  # <- note reverse order and is a multiplication (not an addition as LCU)!
        R_S_matrix=qubit_operator_sparse(R_S_op,n_qubits=N_Qubits)

        R_S_op_dag = hermitian_conjugated(R_S_op)
        R_S_matrix_dag=qubit_operator_sparse(R_S_op_dag,n_qubits=N_Qubits)

        Ps_mat = qubit_operator_sparse(Ps, n_qubits=N_Qubits)

        H_S = QubitOperator()
        for QubitOp in normalised_FULL_set['PauliWords']:
            H_S += QubitOp
        H_S_matrix = qubit_operator_sparse(H_S, n_qubits=N_Qubits)

        RHR = R_S_matrix.dot(H_S_matrix.dot(R_S_matrix_dag))

        if not sparse_allclose(Ps_mat, RHR, atol=atol, rtol=rtol):
            raise ValueError('error in unitary partitioning reduction: R H_s R† != Ps')

    return X_sk_theta_sk_list, normalised_FULL_set, Ps, gamma_l

In [ ]:
key_larg, largest_AC_set = max(anti_commuting_sets_SeqRot.items(), key=lambda x:len(x[1])) 

In [ ]:
n_qubits

In [ ]:
S_index=0
check_reduction=True
Get_Xsk_op_list_NEW(largest_AC_set, S_index, n_qubits, check_reduction=False, atol=1e-8, rtol=1e-05)

In [ ]:
S_index=0
check_reduction=True
Get_Xsk_op_list(largest_AC_set, S_index, n_qubits, check_reduction=False, atol=1e-8, rtol=1e-05)

In [ ]:
import concurrent.futures
from quchem.Unitary_Partitioning.Graph import VectorPauliWord
def fast_qubit_operator_sparse(QubitOp, n_qubits):

    M = csr_matrix((2**n_qubits, 2**n_qubits), dtype=complex)
    for CiPi in QubitOp:
        Pi, Ci = tuple(*CiPi.terms.items())
        p_sym = VectorPauliWord(n_qubits, CiPi).Pvec.todense()
        M+= _fast_symplectic_qubit_operator_sparse(p_sym, Ci)
    
    return M

In [12]:
import concurrent.futures
def parallel_fast_qubit_sparse(QubitOp, n_qubits):
    
    qubit_op_list = list(QubitOp)
    qubit_n_list = list([n_qubits for _ in range(len(qubit_op_list))])
#     input_list = list(zip(qubit_op_list, qubit_n_list))
    
#     op_sparse = lambda q_op, N_q: qubit_operator_sparse(q_op, n_qubits=N_q)
#     
    with concurrent.futures.ProcessPoolExecutor() as exectuor:
        result_list = exectuor.map(fast_qubit_operator_sparse, qubit_op_list, qubit_n_list)
#         result_list = exectuor.map(op_sparse, qubit_op_list, qubit_n_list)
        
    out_matrix = reduce(lambda x,y: x+y, result_list)
    
    return out_matrix
    


In [20]:
OP = QubitOperator('Z1 X9', 12) + QubitOperator('X0 Z1', 1) + QubitOperator('X0 Z9', 1) + QubitOperator('Y0', 1) + QubitOperator('Y0 Y1', 1)
out = parallel_fast_qubit_sparse(OP, 10)
out

<1024x1024 sparse matrix of type '<class 'numpy.complex128'>'
	with 3072 stored elements in Compressed Sparse Row format>

In [21]:
%timeit parallel_fast_qubit_sparse(OP, 10)

50.9 ms ± 665 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%timeit qubit_operator_sparse(OP)

5.06 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
S_index=0
check_reduction=True
key_larg, largest_AC_set = max(anti_commuting_sets_SeqRot.items(), key=lambda x:len(x[1])) 

X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l =Get_Xsk_op_list(largest_AC_set, S_index, n_qubits, check_reduction=False, atol=1e-8, rtol=1e-05)

In [32]:
R_S_q_ops = Get_Rsl_matrix_as_qubitops(X_sk_theta_sk_list)

In [40]:
len(list(R_S_q_ops))

4096

In [36]:
R_dag_P_R = hermitian_conjugated(R_S_q_ops) * Ps * R_S_q_ops

In [38]:
len(list(R_dag_P_R))

4096

In [34]:
%timeit fast_qubit_operator_sparse(R_S_q_ops, n_qubits)

9.09 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit parallel_fast_qubit_sparse(R_S_q_ops, n_qubits)

8.14 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
full_mat = parallel_fast_qubit_sparse(R_S_q_ops, n_qubits)

In [48]:
def Get_Rsl_matrix_as_qubitops_correct_order(Xsk_op_list):

    """
    Function that gives matrix of Rsl from a list of X_sk operators, theta_sks. This is the output from Get_Xsk_op_list function.
    X_sk operators from a given anti_commuting set and S_index

    Args:
        X_sk_theta_sk_list(list): list of tuples containing X_sk QubitOperator and Theta_sk value

    returns:
        R_S_q_ops (QubitOperator)

    """

    ### old SLOW method (exponentiated matrices)
    # R_sk_list = []
    # for X_sk_Op, theta_sk in Xsk_op_list:
    #     pauliword_X_sk_MATRIX = qubit_operator_sparse(QubitOperator(list(X_sk_Op.terms.keys())[0], -1j),
    #                                                   n_qubits=N_Qubits)
    #     const_X_sk = list(X_sk_Op.terms.values())[0]
        
    #     R_sk_list.append(expm(pauliword_X_sk_MATRIX * theta_sk / 2 * const_X_sk))
    # Rs_l_matrix = reduce(np.dot, R_sk_list[::-1])  # <- note reverse order!

    ### new FAST method (symbolic application of rotation operators!)
    R_sk_list = []
    for X_sk_Op, theta_sk in Xsk_op_list:
        op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
        R_sk_list.append(op)

    return R_sk_list[::-1]

In [85]:
ordered_terms = Get_Rsl_matrix_as_qubitops_correct_order(X_sk_theta_sk_list)

In [86]:
def apply_ordered_terms(list_ordered_terms, ground_state_ket,n_qubits, decimal_place_threshold=10):
    sparse_ket = csc_matrix(np.around(ground_state_ket,decimal_place_threshold).reshape([ground_state_ket.shape[0],1]), dtype=complex)
    
    for op in list_ordered_terms:
        sparse_ket = fast_qubit_operator_sparse(ordered_terms[0], n_qubits) @ sparse_ket
    return sparse_ket

In [87]:
new_ket = apply_ordered_terms(ordered_terms,
                    ground_state_ket,
                    n_qubits,
                    decimal_place_threshold=10)

en = new_ket.conj().T @ fast_qubit_operator_sparse(Ps, n_qubits) @ new_ket
en.todense().item(0) * gamma_l

(-0.06266722299162544-1.4940856532829159e-27j)

In [88]:
H_sl = reduce(lambda x,y:x+y, largest_AC_set)

In [89]:
ket =  csc_matrix(np.around(ground_state_ket,10).reshape([ground_state_ket.shape[0],1]), dtype=complex)
ernergy = ket.conj().T @ fast_qubit_operator_sparse(H_sl, n_qubits) @ ket
ernergy.todense().item(0)

(-1.3865741219713452+8.654706546377954e-17j)

In [111]:
energy=0
for op in H_sl:
    energy+= (ket.conj().T @ fast_qubit_operator_sparse(op, n_qubits) @ ket).todense().item(0)
energy

(-1.3865741219713419-5.774569190374004e-17j)

In [128]:
def Apply_Rsl_to_gs_ket(Xsk_op_list, ground_state_ket, n_qubits, decimal_place_threshold=14):

    """
    Given a list of X_sk operators (in correct order), apply R_sk iteratively to ground state ket...
    
    Overall |ψ_out> =  R_s|ψ_ground>  =  R_sk_0 @ R_sk_1 @....|ψ_ground>

    Args:
        X_sk_theta_sk_list(list): list of tuples containing X_sk QubitOperator and Theta_sk value
        ground_state_ket (np.array): 1D numpy array of ground state
        n_qubits (int): number of qubits
        decimal_place_threshold (int): d.p threshold for amplitudes of ground state

    returns:
        ket (csr_matrix): sparse vector, where Rs has been applied to ground state.

    """
    ket =  csc_matrix(np.around(ground_state_ket,decimal_place_threshold).reshape([ground_state_ket.shape[0],1]), dtype=complex)

    for X_sk_Op, theta_sk in Xsk_op_list:
        R_sk_op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op

        ket = fast_qubit_operator_sparse(R_sk_op, n_qubits) @ ket
        

    return ket

In [129]:
def SeqRot_linalg_Energy_iterative(anti_commuting_sets, S_key_dict, N_Qubits,ground_state_ket,
                                   atol=1e-8, rtol=1e-05, decimal_place_threshold=14):
    """
    Function giving ground state energy of Hamiltonian given as a dictionary of anti-commuting sets.
    Note this actually applies R_s to ground state vector then measures the expectation val of P_s

    THIS seems faster than other matrix approaches!

    Args:
        anti_commuting_sets (dict): dictionary of int keys with list of anti commuting QubitOperators sets
        S_key_dict(dict): dictionary keys match that of anti_commuting_sets. Value gives index of P_s operator
        N_Qubits(int): number of qubits
        ground_state_ket (np.array): 1D numpy array of ground state
        decimal_place_threshold (int): d.p threshold for amplitudes of ground state
    returns:
        FCI_Energy(float): Ground state energy

    """
    
    ground_state_ket = csc_matrix(np.around(ground_state_ket,decimal_place_threshold).reshape([ground_state_ket.shape[0],1]), dtype=complex)
    FCI_Energy=0
    for key in anti_commuting_sets:
        AC_set = anti_commuting_sets[key]

        if len(AC_set) < 2:
            matrix_to_measure = fast_qubit_operator_sparse(AC_set[0], n_qubits) 
            active_ket = ground_state_ket.copy()
        else:
            S_index = S_key_dict[key]

            X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, S_index, N_Qubits, check_reduction=check_reduction, atol=atol, rtol=rtol)

            matrix_to_measure = gamma_l * fast_qubit_operator_sparse(Ps, n_qubits) 
            active_ket = Apply_Rsl_to_gs_ket(X_sk_theta_sk_list, 
                                             ground_state_ket,
                                             N_Qubits, 
                                             decimal_place_threshold=decimal_place_threshold)

        exp_val = active_ket.conj().T @ matrix_to_measure @ active_ket
        FCI_Energy+=exp_val.todense().item(0)

    return FCI_Energy

In [122]:
E_SeqRot = SeqRot_linalg_Energy_iterative(anti_commuting_sets_SeqRot,
                             all_zero_Ps_index_dict,
                             n_qubits,
                              ground_state_ket,
                             atol=1e-8,
                             rtol=1e-05)

In [123]:
E_SeqRot

(-133.5425660803483+6.156275230267504e-16j)

In [98]:
ground_state_ket.shape[0] == 2**16

True

In [109]:
output_ket = Apply_Rsl_to_gs_ket(X_sk_theta_sk_list, ground_state_ket, n_qubits)

In [115]:
en = output_ket.conj().T @ fast_qubit_operator_sparse(Ps, n_qubits) @ output_ket
en.todense().item(0) * gamma_l

(-1.3865741220622214-1.5611839779520109e-28j)

In [105]:
type(ground_state_ket)

numpy.ndarray